## <span style="color:red">Costruzione di una foresta di copertura di costo minimo</span>

### La classe completa, inclusi metodi per la visualizzazione e l'animazione dell'algoritmo

In [ ]:
from pygraphviz import AGraph

class graph:
    '''Versione 3.0: include una nuova definizione per gli archi ma il resto dei metodi è immutato
       (come funzionalità
    '''
    class edge(tuple):
        '''edge ora è sottoclasse di tuple. Un edge è dunque una tupla con in più
           l'attributo che memorizza il peso e, come tale, può essere manipolato in modo
           standard proprio come una tupla
        '''
        def __new__(cls, u,v,weight=0):
            '''Per la creazione ridefiniamo il costruttore __new__ della supeclasse
               passando i due estremi in ordine (prima il minore). Se si ridefinisce __new__
               non c'è chiamata automatica a init. L'eventuale chiamata deve essere
               esplicitamente fatta dentro __new__ ma ciò, quasi sempre, è inutile. Possiamo
               infatti mettere le altre inizializzazioni dentro __new__stessa. Nel nostro caso,
               si tratta solo di definire l'attributo che rappresenta il peso
            '''
            self = super().__new__(cls, (min(u,v), max(u,v)))
            self.w = weight
            return self
        
        def __lt__(self,other):
            '''Prestare qui attenzione alla differenza rispetto alla versione 1.0.
               Siccome gli archi sono tuple (e la definizione "built-in" di tupla supporta
               il confronto), possiamo direttamente confrontare due archi anziché confrontare i loro
               (non più necessari) attributi "e" (come nella versione 1.0). Non possiamo però usare
               l'operatore < perché questo produrrebbe una ricorsione infinita. Dobbiamo usare
               il confronto definito nella superclasse.
            '''
            return self.w<other.w or self.w==other.w and super().__lt__(other)
        
    def __init__(self, *args):
        '''Identica alla versione 1.0'''
        self.__edges = set()
        self.__nodes = set()   
        for e in args:
            self.add_edge(*e)
            
    def __str__(self):
        '''Notare la differenza con la 1.0'''
        s = self.to_string()
        return "Weighted graph "+s[s.find('{'):]
    
    def __edges(self):
        '''Identica alla 1.0'''
        return(sorted(self.__edges))
        
    def __explore(self,v):
        '''Returns the set of nodes reachable from v in self'''
        R = {v}
        for u in self.adj_list(v):
            if not self.__visited[u]:
                self.__visited[u] = True
                R = R.union(self.__explore(u))
        return R
        
    def adj_list(self,u):
        '''Returns the sorted list of nodes adjacent to u in self'''
        return sorted([e[1] if e[0]==u else e[0] for e in self.edges if u in e])        
          
    def add_edge(self,u,v,weight=0):
        '''Aggiunge, se non già presente, l'arco (u,v) al grafo.
           Aggiorna l'insieme dei vertici, inserendo anche eventuali vertici isolati.
        '''
        ee = self.edge(u,v,weight)
        
        self.__edges.add(self.edge(u,v,weight))
        self.__nodes = self.__nodes.union({u,v})
        for x in range(1, max(u,v)+1):
            self.__nodes.add(x)
            
    def remove_edge(self,u,v):
        '''Interface function to remove an edge'''
        u,v = min(u,v),max(u,v)
        if v in self.adj_list(u):
            self.__edges.remove((u,v))
            
    def __add_node(self,u):
        '''Class users are not supposed to explicitly add nodes'''
        self.__nodes.add(u)
                   
    def update_graph(self,edge_list):
        '''Called upon a direct assignment to the edge list of the graph.
           It simply create a new graph and copy nodes ad edges to self.
           This way, as long as the value assigned is a legal edge list 
           (possibly with weight) the modified graph is a legal graph.
        '''
        from copy import copy
        g = graph(*edge_list)
        self.__edges=copy(g.__edges)
        self.__nodes=copy(g.__nodes)
        
    def connected(self,u,v):
        '''Returns True iff u is connected to v in self''' 
        if u in self.nodes and v in self.nodes:
            self.__visited = [False]*(len(self.nodes)+1)
            return v in self.__explore(u)
        return False
    
    def __highlight(self,G,E,color):
        '''
        G is the string repr. of a pygraphviz graph, 
        E is a set of edges to be highlighted,
        color is the name of a known (by graphviz) color
        '''
        for e in E:
            i = G.find(str(e[0])+' -- '+str(e[1]))
            j = G.find('[label='+str(e.w),i)
            k = len('[label='+str(e.w))
            l = G.find(']',j+k)
            G = G[:j+k]+',style=bold,color='+color+G[l:]
        return G
    
    def draw(self,highlight=[],color='Red',fn="a_graph"):
        '''Draws the self graph in the fn.pdf file'''
        S = self.to_string()
        if highlight:
            S = self.__highlight(S,highlight,color)
        X = AGraph().from_string(S)
        X.draw(fn+'.pdf',prog='dot',args='-Grankdir=LR')
        
    def to_string(self):
        '''Return a graphviz string representation of self'''
        s = 'strict graph "" {'
        for e in self.edges:
            s += '\n\t'+str(e[0])+' -- '+str(e[1])+'\t['+\
                 'label='+str(e.w)+'];'
        s += '\n}\n'
        return s
    
    def MSF(self):
        '''Computes a minimum spanning forest of self graph'''
        T = graph()
        for u in self.nodes:
            T.__add_node(u)
        for e in self.edges:
            if not T.connected(e[0],e[1]):
                T.add_edge(*e,e.w)
        return T
    
    def animated_MSF(self,fn='mst_animated',pause=5):
        '''Computes a minimum spanning forest of self graph.
           Animated algorihtm. After method invocation,
           open the fn.pdf file with a viewer that updates the view
           upon file changes (e.g. evince)'''
        from time import sleep
        T = graph()
        for u in self.nodes:
            T.__add_node(u)
        S = self.to_string()
        S = self.__highlight(S,self.edges,'Black')
        X = AGraph().from_string(S)
        X.draw(fn+'.pdf',prog='dot',args='-Grankdir=LR')
        sleep(15)
        for e in self.edges:
            S = self.__highlight(S,[e],'Blue')
            X = AGraph().from_string(S)
            X.draw(fn+'.pdf',prog='dot',args='-Grankdir=LR')
            sleep(pause)
            if not T.connected(e[0],e[1]):
                S = self.__highlight(S,[e],'Green')
                T.add_edge(*e,e.w)
            else:
                S = self.__highlight(S,[e],'Red')
            X = AGraph().from_string(S)
            X.draw(fn+'.pdf',prog='dot',args='-Grankdir=LR')
            sleep(pause)
        return T
            
    edges=property(__edges, update_graph)
    nodes=property(lambda self: self.__nodes) # nodes can only be read

### Lettura da file e costruzione di un grafo pesato (per comodità riportiano la definizione di readgraph

In [ ]:
def readgraph(fn):
    '''Legge il grafo da file. Ogni riga deve essere composta da tre numeri:
        i primi due rappresentano i nodi (estremi dell'arco) mentre il terzo
        rappresenta il peso.'''
    G = graph()
    with open(fn) as f:
        for l in f:                           # l è una riga del file, letta come stringa
            tokens = l.strip().split(' ')     # strip() elimina caratteri "sporchi" a fine linea; split()
                                              # restituisce una lista di token (definiti dal separatore spazio)
            u = int(tokens[0])                # Il primo token rappresenta un vertice (deve essere un intero)
            v = int(tokens[1])                # Idem per il secondo
            w = float(tokens[2])              # Il terzo token rappresenta il peso (deve essere un reale)
            G.add_edge(u,v,w)                 # Aggiungiamo l'arco          
    return G

G = readgraph('graph2.txt')

### Costruzione della "foresta"

In [ ]:
F = G.MSF()

In [ ]:
G.edges

In [ ]:
F.edges

### Visualizzazione e animazione (parti non spiegate a lezione)

In [ ]:
G.draw(fn='graph2')

In [ ]:
G.draw(fn='graph2',highlight=F.edges,color='Blue')

In [ ]:
F = G.animated_MSF(fn='graph2')